In [139]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import time
from pynq import MMIO
import multiprocessing as mp

In [140]:
overlay = Overlay("LED.bit")

In [141]:
help(overlay)

Help on Overlay in module pynq.overlay:

<pynq.overlay.Overlay object>
    Default documentation for overlay LED.bit. The following
    attributes are available on this overlay:
    
    IP Blocks
    ----------
    pwm_0                : pynq.overlay.DefaultIP
    axi_gpio_0           : pynq.lib.axigpio.AxiGPIO
    zynq_ultra_ps_e_0    : pynq.overlay.DefaultIP
    
    Hierarchies
    -----------
    None
    
    Interrupts
    ----------
    None
    
    GPIO Outputs
    ------------
    None
    
    Memories
    ------------
    PSDDR                : Memory



In [142]:
def led_ethernet():
    led_top.write(1, mask)
    led_down.write(1, mask)
    time.sleep(1)
    
    while(1):
        led_top.write(0, mask)
        time.sleep(sec)

        if led_down.read() == 1:
            led_down.write(1, mask)
        else:
            led_down.write(1, mask)
            time.sleep(sec)

        led_top.write(2, mask)
        time.sleep(sec)
        led_top.write(3, mask)
        time.sleep(sec) 

        led_down.write(3, mask)
        time.sleep(sec)
        led_top.write(1, mask)
        time.sleep(sec)
        led_down.write(2, mask)
        time.sleep(sec)
        led_down.write(0, mask)
        time.sleep(sec)
        
def led_pwm():
    mmio = MMIO(base_address, address_range)
    mmio.write(REG0, 17179)
    
    while(1):
        for duty in range(17179, 9999999, 5000):
            mmio.write(REG1, duty)
            time.sleep(0.1)

In [ ]:
led_ip = overlay.ip_dict['axi_gpio_0']
pwm_ip = overlay.ip_dict['pwm_0']

led_top = AxiGPIO(led_ip).channel1
led_down = AxiGPIO(led_ip).channel2

base_address = 0xA0000000
address_range = 0x10000
REG0 = 0x00
REG1 = 0x04

mask = 0xf
sec = 0.05

p1 = mp.Process(target=led_ethernet)
p2 = mp.Process(target=led_pwm)

p1.start()
p2.start()

p1.join()
p2.join()
    